<a href="https://colab.research.google.com/github/SoumyaAIDev/Hugging_Face_LLM_models/blob/main/unsloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install unsloth
!pip install --upgrade --no-cache-dir unsloth transformers torch accelerate peft bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 3.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/5


Usage:   
  pip3 <command> [options]

no such option: -U


In [2]:
!pip install torch transformers datasets accelerate unsloth peft bitsandbytes


In [9]:
!pip install --upgrade unsloth transformers
import torch
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import FastLanguageModel

dataset = load_dataset("akshatmehta98/Flipkart-Dataset")

model_name = "unsloth/llama-3-8b"
max_seq_length = 1024

model, tokenizer =FastLanguageModel.from_pretrained(
    model_name =model_name,
    max_seq_length = max_seq_length,
    load_in_4bit=True,
    trust_remote_code=True
)

model =FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "v_proj", "o_proj", "gate_proj", "up_proj" ,"down_proj"],
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing=True,
)

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:195: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.2.12 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [10]:
def format_example(example):
   prompt = f"Generate a product text for: {example['product_name']}\n\n"
   response = example["text"]
   return {"prompt": prompt, "response": response}
train_data = dataset["train"].map(format_example)

def tokenize_function(example):
  inputs = tokenizer(example["prompt"], truncation=True, padding="max_length", max_length=max_seq_length)
  labels = tokenizer(example["response"], truncation=True, padding="max_length", max_length=max_seq_length)
  inputs["labels"] = labels["input_ids"]
  return inputs

tokenized_dataset =train_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

In [11]:
training_args = TrainingArguments(
    output_dir="./llama3-filpkart-finetuned",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    fp16=True,
    logging_dir=".logs",
    logging_steps=10,
    learning_rate=2e-4,
    warmup_steps=100,
    weight_decay=0.01,
    report_to="none",


)

In [12]:
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
    tokenizer=tokenizer,
)
trainer.train


Applying chat template to train dataset (num_proc=2):   0%|          | 0/30000 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/30000 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/30000 [00:00<?, ? examples/s]

<bound method Trainer.train of <unsloth_compiled_cache.UnslothSFTTrainer.UnslothSFTTrainer object at 0x794b2a8fb3d0>>

In [15]:
import torch
from unsloth import FastLanguageModel

def generate_description():
    # Ask the user to enter a product name
    product_name = input("Enter the product name: ")

    # Prepare prompt
    prompt = f"Generate a product text for: {product_name}\n\n"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Call for_inference before generating
    inference_model = FastLanguageModel.for_inference(model)

    # Generate description
    output = inference_model.generate(**inputs, max_new_tokens=100)
    description = tokenizer.decode(output[0], skip_special_tokens=True)

    print("\nGenerated Product Description:")
    print(description)

# Run the function
generate_description()


Enter the product name: Candes 12 L Room/Personal Air Cooler??????(White, Black, Elegant High Speed-Honey Comb Cooling Pad & Ice Chamber, Blower)

Generated Product Description:
Generate a product text for: Candes 12 L Room/Personal Air Cooler??????(White, Black, Elegant High Speed-Honey Comb Cooling Pad & Ice Chamber, Blower)

## Candes 12 L Room/Personal Air Cooler??????(White, Black, Elegant High Speed-Honey Comb Cooling Pad & Ice Chamber, Blower)

## Candes 12 L Room/Personal Air Cooler??????(White, Black, Elegant High Speed-Honey Comb Cooling Pad & Ice Chamber, Blower)

## Candes 12 L Room/Personal Air Cooler??????(White, Black, Elegant High Speed-Honey Comb Cooling Pad & Ice Chamber, Blower)

##


In [16]:
import torch
from unsloth import FastLanguageModel

def generate_review():
    # User input
    product_name = input("Enter the product name: ")

    # Prompt for review-style response
    prompt = f"Write a positive customer review for the product: {product_name}\n\n"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Load fine-tuned model for inference
    inference_model = FastLanguageModel.for_inference(model)

    # Generate response
    output = inference_model.generate(**inputs, max_new_tokens=50)
    review = tokenizer.decode(output[0], skip_special_tokens=True)

    print("\nGenerated Customer Review:")
    print(review)

# Run the function
generate_review()


Enter the product name: Candes 12 L Room/Personal Air Cooler??????(White, Black, Elegant High Speed-Honey Comb Cooling Pad & Ice Chamber, Blower

Generated Customer Review:
Write a positive customer review for the product: Candes 12 L Room/Personal Air Cooler??????(White, Black, Elegant High Speed-Honey Comb Cooling Pad & Ice Chamber, Blower

This review is from: Candes 12 L Room/Personal Air Cooler??????(White, Black, Elegant High Speed-Honey Comb Cooling Pad & Ice Chamber, Blower

Write a positive customer review for the product: Candes 12
